# Self-training as pretraining
The idea is based on the [paper](https://arxiv.org/abs/2206.02909).
The transformation are used as pretext tasks on unlabeled data to learn features.
The learnt weights are used for initialization of weights.
To see different transformation, look into Transformations notebook.
Two types of self-training are tested:
1. binary classification, whether the transformation of time series was applied
2. multi-task classification, which transformation was applied

## Binary classification
Prepare datasets

In [1]:
from cnn1d_modules import Model, Dataset
from transformation import identity, flip, permute, time_warp, negate
from statistics import Statistics

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
dataset_obj = Dataset("pa2")
dataset_obj.read_dataset()
dataset_obj.downsample_dataset()
dataset_obj.segment_dataset()

dataset_obj2 = Dataset("opp")
dataset_obj2.read_dataset()
dataset_obj2.downsample_dataset()
dataset_obj2.segment_dataset()

PATH:  /home/dmitrii/GitHub/SDAPraktikum/E3/PAMAP2_Dataset/pamap2.h5
x_train shape =  (128247, 52)
y_train shape = (128247,)
x_test shape = (22659, 52)
y_test shape = (22659,)
x_train shape(downsampled) =  (42749, 52)
y_train shape(downsampled) = (42749,)
x_test shape(downsampled) = (7553, 52)
y_test shape(downsampled) = (7553,)
[ 1  2  3  4  5  6  7 11 12 13 14 18]
[ 1  2  3  4  5  6  7 11 12 13 14 18]
segmenting signal...
signal segmented.
train_x shape = (3561, 25, 52)
train_y shape = (3561,)
test_x shape = (628, 25, 52)
test_y shape = (628,)
PATH:  /home/dmitrii/GitHub/SDAPraktikum/E3/OpportunityUCIDataset/opportunity.h5
x_train shape =  (700165, 77)
y_train shape = (700165,)
x_test shape = (120516, 77)
y_test shape = (120516,)
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
segmenting signal...
signal segmented.
train_x shape = (63650, 23, 77)
train_y shape = (63650,)
test_x shape = (10955, 23, 77)
test_y shape = (109

(array([[[-9.830e-01, -1.990e-01,  1.190e-01, ...,  2.000e-02,
           4.200e-02,  1.750e-01],
         [-9.860e-01, -2.200e-01,  1.140e-01, ...,  1.700e-02,
           3.100e-02,  1.750e-01],
         [-9.870e-01, -2.270e-01,  1.120e-01, ..., -2.700e-02,
           1.500e-02,  1.750e-01],
         ...,
         [-9.720e-01, -1.420e-01,  6.900e-02, ..., -4.500e-02,
           5.800e-02,  1.750e-01],
         [-9.680e-01, -1.420e-01,  6.400e-02, ..., -1.600e-02,
           4.400e-02,  1.750e-01],
         [-9.650e-01, -1.450e-01,  8.700e-02, ...,  8.700e-02,
           2.400e-02,  1.750e-01]],
 
        [[-9.990e-01, -1.420e-01,  1.190e-01, ..., -4.000e-03,
           9.000e-03,  1.750e-01],
         [-9.710e-01, -1.370e-01,  9.000e-02, ..., -1.300e-02,
           2.000e-02,  1.750e-01],
         [-9.680e-01, -1.530e-01,  6.900e-02, ..., -7.000e-03,
           1.500e-02,  1.750e-01],
         ...,
         [-9.640e-01, -1.050e-01,  1.700e-02, ..., -5.400e-02,
          -3.459e+00,  1

### Train on one pretext task: permute transformation

In [4]:
model2 = Model(dataset_obj2)
model2.build_model(self_training=True, num_transforms=2)
transforms = [identity, permute]
model2.self_train(transforms, 50, 0.0005)

opp
X shape = (?, 1, 23, 77)
Y shape = (?, 2)
hidden layer 1 shape
W_conv1 shape = (1, 7, 77, 128)
b_conv1 shape = (128,)
h_conv1 shape = (?, 1, 17, 128)
h_pool1 shape = (?, 1, 6, 128)
hidden layer 2 shape
W_conv2 shape = (1, 3, 128, 128)
b_conv2 shape = (128,)
h_conv2 shape = (?, 1, 4, 128)
h_pool2 shape = (?, 1, 1, 128)
shape's shape: [None, 1, 1, 128]
c_flat shape = (?, 128)
unique test_y [0 1]
unique train_y [0 1]
test_y[1]= [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
train_y shape(1-hot) = (63650, 18)
test_y shape(1-hot) = (10955, 18)
train_x_reshaped =  (63650, 1, 23, 77)
test_x_reshaped =  (10955, 1, 23, 77)
train_x shape = (63650, 23, 77)
train_y shape = (63650,)
test_x shape = (10955, 23, 77)
test_y shape = (10955,)


/home/dmitrii/GitHub/SDAPraktikum/E3/transformation.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  splitted = np.array(np.split(sample_i, np.append(segments, sample.shape[1])))


Epoch:  0  Training Loss:  0.6380585195792753  Training Accuracy:  0.72221524
Epoch:  1  Training Loss:  0.4761764254398509  Training Accuracy:  0.7453417
Epoch:  2  Training Loss:  0.4446214212318301  Training Accuracy:  0.7717361
Epoch:  3  Training Loss:  0.4099332014214945  Training Accuracy:  0.7885153
Epoch:  4  Training Loss:  0.38423654918279687  Training Accuracy:  0.80180675
Epoch:  5  Training Loss:  0.37304419479979356  Training Accuracy:  0.7974077
Epoch:  6  Training Loss:  0.36196343050576313  Training Accuracy:  0.81803614
Epoch:  7  Training Loss:  0.35363956540073427  Training Accuracy:  0.8177219
Epoch:  8  Training Loss:  0.34420418985547796  Training Accuracy:  0.817282
Epoch:  9  Training Loss:  0.3356748224279171  Training Accuracy:  0.8100707
Epoch:  10  Training Loss:  0.33293109525827097  Training Accuracy:  0.8298036
Epoch:  11  Training Loss:  0.32748651072397916  Training Accuracy:  0.820817
Epoch:  12  Training Loss:  0.3248609694084471  Training Accuracy:

In [5]:
model = Model(dataset_obj)
stats = Statistics()

In [6]:
num_experiments = 5

for i in range(num_experiments):
    model.W_conv2_weight = model2.W_conv2_weight
    model.b_conv2_weight = model2.b_conv2_weight
    model.build_model(self_training=False, load_weights=True)
    model.run_model(training_epochs=100, learning_rate=0.0001)
    test_accuracy, val_accuracy, y_pred, y_true = model.test()
    stats.add(test_accuracy, val_accuracy, y_pred, y_true)

pa2
X shape = (?, 1, 25, 52)
Y shape = (?, 11)
hidden layer 1 shape
W_conv1 shape = (1, 7, 52, 128)
b_conv1 shape = (128,)
h_conv1 shape = (?, 1, 19, 128)
h_pool1 shape = (?, 1, 7, 128)
WEIGHTS ARE LOADED
WEIGHTS ARE LOADED
hidden layer 2 shape
W_conv2 shape = (1, 3, 128, 128)
b_conv2 shape = (128,)
h_conv2 shape = (?, 1, 5, 128)
h_pool2 shape = (?, 1, 2, 128)
shape's shape: [None, 1, 2, 128]
c_flat shape = (?, 256)
unique test_y [0 1]
unique train_y [0 1]
test_y[1]= [1 0 0 0 0 0 0 0 0 0 0]
train_y shape(1-hot) = (3561, 11)
test_y shape(1-hot) = (628, 11)
train_x_reshaped =  (3561, 1, 25, 52)
test_x_reshaped =  (628, 1, 25, 52)
train_x shape = (3561, 25, 52)
train_y shape = (3561,)
test_x shape = (628, 25, 52)
test_y shape = (628,)
Epoch:  0  Training Loss:  2.5241180593317205  Training Accuracy:  0.108958155
Epoch:  1  Training Loss:  2.3758114554665304  Training Accuracy:  0.121595055
Epoch:  2  Training Loss:  2.338084743239663  Training Accuracy:  0.14995788
Epoch:  3  Training Los

In [7]:
stats.show()

Test accuracy. Mean (std) = 0.872 (0.015)
Val accuracy. Mean (std) = 0.872 (0.015)
f1 score weighted. Mean (std) = 0.869 (0.016)
f1 score mean. Mean (std) = 0.858 (0.016)


## Multi-class classification: flip, permute, time warp

In [8]:
stats3 = Statistics()
model3 = Model(dataset_obj2)
model3.build_model(self_training=True, num_transforms=4)
transforms = [identity, flip, permute, time_warp]
model3.self_train(transforms, 50, 0.0005)

opp
X shape = (?, 1, 23, 77)
Y shape = (?, 4)
hidden layer 1 shape
W_conv1 shape = (1, 7, 77, 128)
b_conv1 shape = (128,)
h_conv1 shape = (?, 1, 17, 128)
h_pool1 shape = (?, 1, 6, 128)
hidden layer 2 shape
W_conv2 shape = (1, 3, 128, 128)
b_conv2 shape = (128,)
h_conv2 shape = (?, 1, 4, 128)
h_pool2 shape = (?, 1, 1, 128)
shape's shape: [None, 1, 1, 128]
c_flat shape = (?, 128)
unique test_y [0 1]
unique train_y [0 1]
test_y[1]= [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
train_y shape(1-hot) = (63650, 18)
test_y shape(1-hot) = (10955, 18)
train_x_reshaped =  (63650, 1, 23, 77)
test_x_reshaped =  (10955, 1, 23, 77)
train_x shape = (63650, 23, 77)
train_y shape = (63650,)
test_x shape = (10955, 23, 77)
test_y shape = (10955,)
Epoch:  0  Training Loss:  1.4693549322170512  Training Accuracy:  0.2492066
Epoch:  1  Training Loss:  1.3923014126553142  Training Accuracy:  0.2522388
Epoch:  2  Training Loss:  1.3902625898480176  Training Accuracy:  0.24981932
Epoch:  3  Training Loss:  1.3892356820

In [12]:
for i in range(num_experiments):
    model.W_conv2_weight = model3.W_conv2_weight
    model.b_conv2_weight = model3.b_conv2_weight
    model.build_model(self_training=False, load_weights=True)
    model.run_model(training_epochs=100, learning_rate=0.0001)
    test_accuracy, val_accuracy, y_pred, y_true = model.test()
    stats3.add(test_accuracy, val_accuracy, y_pred, y_true)

pa2
X shape = (?, 1, 25, 52)
Y shape = (?, 11)
hidden layer 1 shape
W_conv1 shape = (1, 7, 52, 128)
b_conv1 shape = (128,)
h_conv1 shape = (?, 1, 19, 128)
h_pool1 shape = (?, 1, 7, 128)
WEIGHTS ARE LOADED
WEIGHTS ARE LOADED
hidden layer 2 shape
W_conv2 shape = (1, 3, 128, 128)
b_conv2 shape = (128,)
h_conv2 shape = (?, 1, 5, 128)
h_pool2 shape = (?, 1, 2, 128)
shape's shape: [None, 1, 2, 128]
c_flat shape = (?, 256)
unique test_y [0 1]
unique train_y [0 1]
test_y[1]= [1 0 0 0 0 0 0 0 0 0 0]
train_y shape(1-hot) = (3561, 11)
test_y shape(1-hot) = (628, 11)
train_x_reshaped =  (3561, 1, 25, 52)
test_x_reshaped =  (628, 1, 25, 52)
train_x shape = (3561, 25, 52)
train_y shape = (3561,)
test_x shape = (628, 25, 52)
test_y shape = (628,)
Epoch:  0  Training Loss:  2.5631681290539827  Training Accuracy:  0.1151362
Epoch:  1  Training Loss:  2.388054453242909  Training Accuracy:  0.13086212
Epoch:  2  Training Loss:  2.313029967654835  Training Accuracy:  0.17523168
Epoch:  3  Training Loss:  

In [13]:
stats3.show()

Test accuracy. Mean (std) = 0.852 (0.020)
Val accuracy. Mean (std) = 0.852 (0.020)
f1 score weighted. Mean (std) = 0.847 (0.021)
f1 score mean. Mean (std) = 0.833 (0.023)


## Conclusions
1. Self-training facilitates faster convergence for (num_epochs = 50)
2. No visible improvements for binary or multi-task self-training
3. Possible reason: the tested network is too small
